<a href="https://colab.research.google.com/github/MiguelBetti/Cartographie/blob/main/CartographieLope_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

J'installe *Spacy*.


In [10]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 52.3 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.1 which is incompatible.


Je télécharge le modèle multi-langue, qui permet l'utilisation de ce modèle pour d'autres langues.

In [11]:
!python -m spacy download xx_ent_wiki_sm

2023-03-10 17:02:19.142200: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 17:02:20.364429: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:02:20.364571: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:02:20.364608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

Je télécharges des textes annotés pour l'entraînement, dans un nouveau dossier.

In [12]:
!mkdir training_data
!wget https://raw.githubusercontent.com/MiguelBetti/Cartographie/main/SPACY/DATA/train/LOPE_testtrain.tsv
!mv /content/LOPE_testtrain.tsv /content/training_data


mkdir: cannot create directory ‘training_data’: File exists
--2023-03-10 17:02:37--  https://raw.githubusercontent.com/MiguelBetti/Cartographie/main/SPACY/DATA/train/LOPE_testtrain.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5154 (5.0K) [text/plain]
Saving to: ‘LOPE_testtrain.tsv’

LOPE_testtrain.tsv  100%[===================>]   5.03K  --.-KB/s    in 0s      

2023-03-10 17:02:37 (52.4 MB/s) - ‘LOPE_testtrain.tsv’ saved [5154/5154]



Je visualise l'information dans un dataframe.

In [13]:
!pip install pandas
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
df = pd.read_csv('/content/training_data/LOPE_testtrain.tsv', sep="\t", header=None)
df.head(11)

,0,1
0,Cayó,O
1,la,O
2,gran,O
3,princesa,O
4,de,O
5,Bretaña,B-LOC
6,Nunca,O
7,midieras,O
8,la,O
9,arenosa,O


Je télécharge un autre texte annoté pour la dévolution et je le visualise.

---



In [15]:
!wget https://raw.githubusercontent.com/MiguelBetti/Cartographie/main/SPACY/DATA/dev/LOPE_testdev.tsv
!mv /content/LOPE_testdev.tsv /content/training_data

--2023-03-10 17:02:41--  https://raw.githubusercontent.com/MiguelBetti/Cartographie/main/SPACY/DATA/dev/LOPE_testdev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5027 (4.9K) [text/plain]
Saving to: ‘LOPE_testdev.tsv’

LOPE_testdev.tsv    100%[===================>]   4.91K  --.-KB/s    in 0s      

2023-03-10 17:02:41 (61.7 MB/s) - ‘LOPE_testdev.tsv’ saved [5027/5027]



In [16]:
df = pd.read_csv('/content/training_data/LOPE_testdev.tsv', sep="\t", header=None)
df.head(11)

,0,1
0,Yo,O
1,hice,O
2,salva,O
3,en,O
4,presencia,O
5,de,O
6,la,O
7,hermosura,O
8,del,O
9,alba,O


Je transforme les fichiers .tsv en .json pour pouvoir les manipuler avec Spacy.

In [17]:
!mkdir training_json
!python -m spacy convert /content/training_data/LOPE_testtrain.tsv /content/training_json -t json -n 10 -c iob 
!python -m spacy convert /content/training_data/LOPE_testdev.tsv /content/training_json -t json -n 10 -c iob -b xx_ent_wiki_sm

mkdir: cannot create directory ‘training_json’: File exists
2023-03-10 17:02:44.809268: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 17:02:46.447131: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:02:46.447299: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:02:46.447327: W tens

J'affiche un extrait du résultat.

In [18]:
f = open('/content/training_json/LOPE_testtrain.json')
content = f.readlines()
print(*content[74:93])

                "orth":"arenosa",
                 "space":" ",
                 "tag":"-",
                 "ner":"O"
               },
               {
                 "id":10,
                 "orth":"orilla",
                 "space":" ",
                 "tag":"-",
                 "ner":"O"
               },
               {
                 "id":11,
                 "orth":"con",
                 "space":" ",
                 "tag":"-",
                 "ner":"O"
               },



Je transforme mon fichier .json dans le format spécifique de Spacy.

In [19]:
!mkdir training_spacy
!python -m spacy convert /content/training_json/LOPE_testtrain.json /content/training_spacy -t spacy
!python -m spacy convert /content/training_json/LOPE_testdev.json /content/training_spacy -t spacy

2023-03-10 17:03:12.155837: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 17:03:13.723508: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:03:13.723670: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:03:13.723697: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

J'installe la bibliothèque NVIDIA 9.2 cuda.

In [20]:
!wget  	https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-11-3_11.3.0-1_amd64.deb -O cuda-11-3_11.3.0-1_amd64.deb
!dpkg -i cuda-11-3_11.3.0-1_amd64.deb
!apt-key add /var/cuda-11-3-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-11-3
!nvcc --version

--2023-03-10 17:03:31--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-11-3_11.3.0-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2448 (2.4K) [application/x-deb]
Saving to: ‘cuda-11-3_11.3.0-1_amd64.deb’

cuda-11-3_11.3.0-1_ 100%[===================>]   2.39K  --.-KB/s    in 0s      

2023-03-10 17:03:32 (98.5 MB/s) - ‘cuda-11-3_11.3.0-1_amd64.deb’ saved [2448/2448]

Selecting previously unselected package cuda-11-3.
(Reading database ... 128275 files and directories currently installed.)
Preparing to unpack cuda-11-3_11.3.0-1_amd64.deb ...
Unpacking cuda-11-3 (11.3.0-1) ...
dpkg: dependency problems prevent configuration of cuda-11-3:
 cuda-11-3 depends on cuda-runtime-11-3 (>= 11.3.0); however:
  Package cuda-runtime-11-3 is not insta

J'installe PyTorch pour le machine learning.

In [21]:
!pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 956.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.13.1+cu116
    Uninstalling torchaudio-0.13.1+cu116:
      Successfully uninstalled torchaudio-0.13.1+cu116
ERROR: pip's dependency resolver does not

J'installe les transformeurs Spacy et je modifie le path.

In [22]:
!pip install -U spacy[cuda113,transformers]
!export CUDA_PATH=”/usr/local/cuda-11.3"
!export LD_LIBRARY_PATH=$CUDA_PATH/lib64:$LD_LIBRARY_PATH

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 16.0 MB/s eta 0:00:00
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


J'installe cupy, pour manipuler des matrices optimisées pour les GPU.

In [23]:
!pip install cupy-cuda113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Je configure mon entraînement avec un fichier .cfg qui contient tous les paramètres pour mon entraînement.

In [24]:
!wget https://raw.githubusercontent.com/MiguelBetti/Cartographie/main/SPACY/DATA/config.cfg

--2023-03-10 17:08:34--  https://raw.githubusercontent.com/MiguelBetti/Cartographie/main/SPACY/DATA/config.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1778 (1.7K) [text/plain]
Saving to: ‘config.cfg’

config.cfg          100%[===================>]   1.74K  --.-KB/s    in 0s      

2023-03-10 17:08:34 (22.8 MB/s) - ‘config.cfg’ saved [1778/1778]



Je reconfigure l'entraînement.

In [25]:
!python -m spacy init fill-config /content/config.cfg /content/config_spacy.cfg

2023-03-10 17:08:39.821809: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 17:08:43.363575: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:08:43.363785: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:08:43.363817: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

Je contrôle mon fichier de configuration.

In [26]:
!python -m spacy debug data /content/config_spacy.cfg

2023-03-10 17:09:14.899884: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 17:09:15.924098: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:09:15.924230: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:09:15.924253: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

Je contrôle que j'ai bien un GPU à dispo

In [27]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e9682313-a896-8562-9e05-a950512cb601)


Je lance mon rentraînement.

In [28]:
!python -m spacy train -g 0 /content/config_spacy.cfg --output ./

2023-03-10 17:12:42.044463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 17:12:43.144745: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:12:43.144908: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:12:43.144932: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

Je télécharge mon meilleur modèle.

In [29]:
from google.colab import files
!zip -r /content/model-best.zip /content/model-best
files.download('/content/model-best.zip')

  adding: content/model-best/ (stored 0%)
  adding: content/model-best/ner/ (stored 0%)
  adding: content/model-best/ner/model (deflated 8%)
  adding: content/model-best/ner/cfg (deflated 33%)
  adding: content/model-best/ner/moves (deflated 36%)
  adding: content/model-best/transformer/ (stored 0%)
  adding: content/model-best/transformer/model (deflated 8%)
  adding: content/model-best/transformer/cfg (stored 0%)
  adding: content/model-best/tokenizer (deflated 84%)
  adding: content/model-best/config.cfg (deflated 61%)
  adding: content/model-best/vocab/ (stored 0%)
  adding: content/model-best/vocab/vectors (deflated 45%)
  adding: content/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/model-best/vocab/key2row (stored 0%)
  adding: content/model-best/vocab/strings.json (deflated 72%)
  adding: content/model-best/vocab/lookups.bin (stored 0%)
  adding: content/model-best/meta.json (deflated 58%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Je teste le modèle sur un nouveau texte. Pour cela, je télécharge et transforme le nouveau document, comme les précédents.

In [30]:
!wget https://raw.githubusercontent.com/MiguelBetti/Cartographie/main/SPACY/DATA/eval/LOPE_AMANTESAMOR.tsv
!mv /content/LOPE_AMANTESAMOR.tsv /content/training_data

--2023-03-10 17:27:59--  https://raw.githubusercontent.com/MiguelBetti/Cartographie/main/SPACY/DATA/eval/LOPE_AMANTESAMOR.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 124909 (122K) [text/plain]
Saving to: ‘LOPE_AMANTESAMOR.tsv’

LOPE_AMANTESAMOR.ts 100%[===================>] 121.98K  --.-KB/s    in 0.02s   

2023-03-10 17:27:59 (5.93 MB/s) - ‘LOPE_AMANTESAMOR.tsv’ saved [124909/124909]



In [42]:
df = pd.read_csv('/content/training_data/LOPE_AMANTESAMOR.tsv', sep="\t", header=None)
df.head(600)

,0
0,Casta O
1,mujer O
2,para O
3,casta O
4,lo O
...,...
595,criada O
596,No O
597,sientes O
598,las O


In [34]:
!mkdir training_json
!python -m spacy convert /content/training_data/LOPE_AMANTESAMOR.tsv /content/training_json -t json -n 10 -c iob 

mkdir: cannot create directory ‘training_json’: File exists
2023-03-10 17:48:52.310861: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 17:48:53.412837: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:48:53.412986: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:48:53.413009: W tens

In [35]:
!mkdir training_spacy
!python -m spacy convert /content/training_json/LOPE_AMANTESAMOR.json /content/training_spacy -t spacy

mkdir: cannot create directory ‘training_spacy’: File exists
2023-03-10 17:49:06.114509: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 17:49:07.198639: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:49:07.198773: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:49:07.198797: W ten

Je teste le modèle avec le nouveau fichier.

In [36]:
!mkdir results
!python -m spacy evaluate /content/model-best /content/training_spacy/LOPE_AMANTESAMOR.spacy  -dp /content/results

2023-03-10 17:49:17.411927: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 17:49:18.407735: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:49:18.407867: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-10 17:49:18.407888: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War